<a href="https://colab.research.google.com/github/Binaya824/layoutLMv3_model_train_on_custom_dataset/blob/main/UBIAI_DATASET_LAYOUTLMV3_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn pandas Pillow numpy torch evaluate seqeval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import zipfile
import argparse
import shutil


# Define path to your ZIP file in Google Drive
zip_file_path = '/content/drive/MyDrive/invoce_dataset_label_studio.zip'
dir_path='/content/data'
if os.path.exists(dir_path):
        # Remove the directory and all its contents
  shutil.rmtree(dir_path)
  print(f"The directory '{dir_path}' and all its contents have been deleted.")
else:
  print(f"The directory '{dir_path}' does not exist.")
# Extract the ZIP file to /content/data
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

# Now continue with the rest of your code...


The directory '/content/data' and all its contents have been deleted.


In [ ]:
import pandas as pd
import os
from datasets.features import ClassLabel
from transformers import AutoProcessor, AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D, Dataset
from datasets import Image as Img
from PIL import Image
import evaluate
import numpy as np
from typing import List, Mapping

import warnings
warnings.filterwarnings('ignore')


def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return f.readlines()


def prepare_examples(examples):
    images = examples[image_column_name]
    words = examples[text_column_name]
    boxes = examples[boxes_column_name]
    word_labels = examples[label_column_name]

    encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                         truncation=True, padding="max_length")
    return encoding


def get_zip_dir_name():
    try:
        # Change directory to where the data is extracted
        os.chdir('/content/data')
        dir_list = os.listdir()
        any_file_name = dir_list[0]
        zip_dir_name = any_file_name.split('_')[0]  # Assuming '_' is the separator in filenames

        if all(list(map(lambda x: x.startswith(zip_dir_name), dir_list))):
            print(f"Zip directory name is {zip_dir_name}")
            return zip_dir_name
        return False
    finally:
        os.chdir('/content')


def filter_out_unannotated(example):
    tags = example['ner_tags']
    return not all([tag == label2id['O'] for tag in tags])


def compute_metrics(p, labels_ref: List[str]) -> Mapping:
    """Trainer function to compute evaluation metrics."""
    metric = evaluate.load("seqeval")

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Keep only labeled tokens
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Transform into string labels
    true_predictions_str = [[labels_ref[p] for p in prediction] for prediction in true_predictions]
    true_labels_str = [[labels_ref[l] for l in label] for label in true_labels]

    # Convert into the BIO format for the evaluation
    BIO_predictions = [["B-" + label if label != "O" else label for label in labels] for labels in true_predictions_str]
    BIO_labels = [["B-" + label if label != "O" else label for label in labels] for labels in true_labels_str]

    # Compute evaluation metrics
    results = metric.compute(predictions=BIO_predictions, references=BIO_labels)

    # # For better logging (optional)
    # LOGGER.info(f"Labels for confusion matrix: {[label for batch in true_labels for label in batch]}")
    # LOGGER.info(f"Preds for confusion matrix: {[pred for batch in true_predictions for pred in batch]}")

    return results


# Define arguments manually for Google Colab
class Args:
    valid_size = 0.1  # Example: 10% validation set
    output_path = '/content/layoutlmv3-finetuned-label-studio-dataset-invoice-0'  # Path to save the output in Colab
    num_epochs = 40
    learning_rate = 5e-5
    batch_size = 8


args = Args()  # Instantiate the arguments

# Make sure output directory exists
os.makedirs(args.output_path, exist_ok=True)

# Load files
files = {}
zip_dir_name = get_zip_dir_name()

if zip_dir_name:
  files['train_box'] = read_text_file(os.path.join(
  '/content/data', f'{zip_dir_name}/{zip_dir_name}_box.txt'))
  files['train_image'] = read_text_file(os.path.join(
  '/content/data', f'{zip_dir_name}/{zip_dir_name}_image.txt'))
  files['train'] = read_text_file(os.path.join(
  '/content/data', f'{zip_dir_name}/{zip_dir_name}.txt'))
else:
  for f in os.listdir('/content/data'):
    if f.endswith('.txt') and 'box' in f:
      files['train_box'] = read_text_file(os.path.join('/content/data', f))
    elif f.endswith('.txt') and 'image' in f:
      files['train_image'] = read_text_file(os.path.join('/content/data', f))
    elif f.endswith('.txt') and 'labels' not in f:
      files['train'] = read_text_file(os.path.join('/content/data', f))

assert len(files['train']) == len(files['train_box'])
assert len(files['train_box']) == len(files['train_image'])
assert len(files['train_image']) == len(files['train'])

# Organize data into format
images = {}
for i, row in enumerate(files['train_image']):
    if row != '\n':
        image_name = row.split('\t')[-1]
        images.setdefault(image_name.replace('\n', ''), []).append(i)

words, bboxes, ner_tags, image_path = [], [], [], []
for image, rows in images.items():
    words.append([row.split('\t')[0].replace('\n', '') for row in files['train'][rows[0]:rows[-1]+1]])
    ner_tags.append([row.split('\t')[1].replace('\n', '') for row in files['train'][rows[0]:rows[-1]+1]])
    bboxes.append([box.split('\t')[1].replace('\n', '') for box in files['train_box'][rows[0]:rows[-1]+1]])
    if zip_dir_name:
        image_path.append(f"/content/data/{zip_dir_name}\\{image}")
    else:
        image_path.append(f"/content/data/{image}")

print(f'Number of images: {image_path}')
image_path = [path.replace('\\', '/') for path in image_path]

labels = list(set([tag for doc_tag in ner_tags for tag in doc_tag]))
if 'O' not in labels:
    labels.append('O')

# Sort the labels for consistency
labels = sorted(labels)

print(f'Labels: {labels}')

# Map labels to ids and vice versa
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in id2label.items()}

# Print for debugging
print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

dataset_dict = {
    'id': range(len(words)),
    'tokens': words,
    'bboxes': [[list(map(int, bbox.split())) for bbox in doc] for doc in bboxes],
    'ner_tags': [[label2id[tag] for tag in ner_tag] for ner_tag in ner_tags],
    'image': [Image.open(path).convert("RGB") for path in image_path]
}

# Define raw features
features = Features({
    'id': Value(dtype='string', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
    'ner_tags': Sequence(feature=ClassLabel(num_classes=len(labels), names=labels, names_file=None, id=None), length=-1, id=None),
    'image': Img(decode=True, id=None)
})

full_data_set = Dataset.from_dict(dataset_dict, features=features)
dataset = full_data_set.train_test_split(test_size=args.valid_size)
dataset["train"] = dataset["train"].filter(filter_out_unannotated)

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

# Set column names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# Define custom features
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

# Prepare datasets
train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
    features=features,
)

eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=dataset["test"].column_names,
    features=features,
)

train_dataset.set_format("torch")
eval_dataset.set_format("torch")

# Save datasets to disk
train_dataset.save_to_disk(f'{args.output_path}/train_split')
eval_dataset.save_to_disk(f'{args.output_path}/eval_split')
dataset.save_to_disk(f'{args.output_path}/raw_data')

# Initialize the LayoutLMv3 model
model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# Define training arguments
training_args = TrainingArguments(
    output_dir=args.output_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=args.learning_rate,
    per_device_train_batch_size=args.batch_size,
    per_device_eval_batch_size=args.batch_size,
    num_train_epochs=args.num_epochs,
    weight_decay=0.01,
    logging_dir=f'{args.output_path}/logs',
    logging_steps=10,
    load_best_model_at_end=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    compute_metrics=lambda p: compute_metrics(p, labels)
)

# Train the model
trainer.train()

# Save the final model
trainer.save_model(f'{args.output_path}/final_model')
# /content/data/ubiai dataset invoice/page_43_image_0.jpg

Number of images: ['/content/data/page_300.jpg', '/content/data/page_228.jpg', '/content/data/page_121.jpg', '/content/data/page_280.jpg', '/content/data/page_273.jpg', '/content/data/page_307.jpg', '/content/data/page_102.jpg', '/content/data/page_85.jpg', '/content/data/page_214.jpg', '/content/data/page_50.jpg', '/content/data/page_424.jpg', '/content/data/page_342.jpg', '/content/data/page_213.jpg', '/content/data/page_142.jpg', '/content/data/page_332.jpg', '/content/data/page_240.jpg', '/content/data/page_238.jpg', '/content/data/page_178.jpg', '/content/data/page_32.jpg', '/content/data/page_183.jpg', '/content/data/page_112.jpg', '/content/data/page_46.jpg', '/content/data/page_304.jpg', '/content/data/page_156.jpg', '/content/data/page_55.jpg', '/content/data/page_309.jpg', '/content/data/page_170.jpg', '/content/data/page_211.jpg', '/content/data/page_161.jpg', '/content/data/page_423.jpg', '/content/data/page_313.jpg', '/content/data/page_36.jpg', '/content/data/page_368.jpg

Filter:   0%|          | 0/391 [00:00<?, ? examples/s]

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub("layoutlmv3-model-ubiai-finetuned")

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

events.out.tfevents.1728368534.17f33622ae76.757.1:   0%|          | 0.00/37.2k [00:00<?, ?B/s]

...

data-00000-of-00001.arrow:   0%|          | 0.00/6.53M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/56.4M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/57.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/riteshbehera857/layoutlmv3-finetuned-ubiai-dataset-invoice-6/commit/b4392be8f208693bd8b8b704954208bc73ec83a3', commit_message='layoutlmv3-model-ubiai-finetuned', commit_description='', oid='b4392be8f208693bd8b8b704954208bc73ec83a3', pr_url=None, pr_revision=None, pr_num=None)